# Create a forge and execute an evolution cycle
In this notebook, we will go through the very few steps needed to run a forge cycle for a given budget. 
Before running this notebook, you must set up a few things:

**Your access to LLM's providers:**

To try Ebiose, you must have access to at least one LLM provider. To provide the credentials, copy-paste the [`model_endpoints_template.yml`](./../model_endpoints_template.yml) file, rename it as `model_endpoints.yml` and fill it with your own credentials.

**Add the root directory to your Python path**

Depending on your settings, you may need to add the root of the repository to your `PYTHONPATH` environment variable. You may also use a `.env` file to do so. copy-paste the [`.env.template`](./../.env.template) file, rename it as `.env` and fill it with your own root directory.

**Use LangFuse for tracing**

Ebiose has chosen LangFuse to provide easy and free observability, through its self-hosted capability. Refer to [Langfuse official documentation](https://langfuse.com/self-hosting) to set it up. Once done, fill `LANGFUSE_SECRET_KEY`, `LANGFUSE_PUBLIC_KEY` and `LANGFUSE_HOST` in the `.env` file.

Other observability tools might be used but are not configured yet.

**Load .env file**

To load the `.env` file, execute:

In [ ]:
from dotenv import load_dotenv
load_dotenv()

## Creating a basic forge

In ebiose, a **forge** is where custom agents are created to solve specific problems. The forge is the exclusive origin of new agents. Within each forge, architects agents orchestrate the creation and improvement of agents by reusing existing building blocks from the ecosystem.

To create a forge and run a cycle, you must provide the following:
- a description of the forge, which defines the problem that must be solved by generated agents;
- the expected format of the agent's input and output, defined as Pydantic models;
- an implementation of the `compute_fitness` abstract method that will be used by the forge to evaluate the generated agents.

Let's say we wich to generate agents specialized in solving math problems. The forge description could be:

In [ ]:
forge_description = "Solving math word problems"

Next, we need to define the expected input and output formats of the generated agents. These formats are to be defined as Pydantic models. 

For instance, in our context of solving math problems, we want the agent input to be a string which will represent the math problem to be solved and the agent output to be composed of two fields:
- `solution` which will be the final solution to the math problem, given as an integer;
- `rationale` which will be the rationale behind the found solution.

The IO Pydantic models will thus be:

In [ ]:
from pydantic import BaseModel

class AgentInput(BaseModel):
        math_problem: str

class AgentOutput(BaseModel):
    solution: int
    rationale: str

Lastly, we must provide a way of evaluating the generated agents through the implementation of the `compute_fitness` abstract method of `AgentForge` class. For the sake of demonstration, we will here return a random float between 0 and 1, so that we don't spend tokens at evaluation.

In [ ]:
import random
random.seed(11)

from ebiose.core.agent import Agent
from ebiose.core.agent_forge import AgentForge

class BasicForge(AgentForge):
    async def compute_fitness(self, agent: Agent, compute_token_id: str, **kwargs: dict[str, any]) -> float:
        return random.random()

We can now instantiate the forge with the provided elements:

In [ ]:
forge = BasicForge(
    name="Basic forge",
    description=forge_description,
    agent_input_model=AgentInput,
    agent_output_model=AgentOutput,
    default_generated_agent_engine_type="langgraph_engine",
    default_model_endpoint_id=None, # set to different model endpoint id if 
    # you want to use a specific model for generated agents, other than the default one defined in the model endpoint YAML file
)

## Running a forge cycle

Once the forge is instantiated, we can start generating agents by running a **forge cycle**. 

To do so, you must define:
- a budget in dollars (the forge cycle will end once this budget is exhausted);
- optionally, a path in which created agents and fitness will be saved accross generations. 

> ⚠️ Note that we need to use `asyncio.run` to launch the forge cycle.

> 🚨 Before executing the following cell, check the amount of budget you have allocated!

> 💡 If you are using VSCode, install the [*Markdown Preview Mermaid Support* extension](https://marketplace.visualstudio.com/items?itemName=bierner.markdown-mermaid) to allow the display of the generated agent's graphs.

In [ ]:
import asyncio
import nest_asyncio

from ebiose.core.evo_forging_cycle import EvoForgingCycleConfig
nest_asyncio.apply()

from pathlib import Path
from datetime import UTC, datetime

# the path where results will be saved
current_time = datetime.now(UTC).strftime("%Y-%m-%d_%H-%M-%S")
SAVE_PATH = Path(f"./../data/") / current_time
if not SAVE_PATH.exists():
    SAVE_PATH.mkdir(parents=True)


cycle_config = EvoForgingCycleConfig(
    budget=1.0, # the budget for the forge cycle in dollas
    n_agents_in_population=4, # number of agents in the population, at each generation
    n_selected_agents_from_ecosystem=0, # number of agents selected from the ecosystem at initialization
    n_best_agents_to_return=2, # number of best agents to return at the end of the cycle
    replacement_ratio=0.5, # ratio of agents replaced at each generation
    save_path=SAVE_PATH, # the path where results will be saved (optional)
    node_types = ["StartNode", "EndNode", "LLMNode"],
)

best_agents, best_fitness = asyncio.run(
    forge.run_new_cycle(config=cycle_config)
)


We can now display the best agents that have been returned as follows. Note that:
- all agents can be found in the `SAVE_PATH` directory if you defined one;
- here, the compute fitness only returns a random float, so the following displayed agents have not been truly evaluated. 

Go check [examples/math_forge/math_forge.py](./../examples/math_forge/math_forge.py) to see a fully implemeted forge with a non-random fitness evaluation function.

## Display best agents

In [ ]:
forge.display_results(best_agents, best_fitness)